# Model Comparison and Analysis

This notebook compares model fits and analyzes parameter stability.

**Contents:**
1. SVI Parameter Analysis
2. Fit Quality Over Time
3. Term Structure Evolution
4. Parameter Stability
5. Out-of-Sample Analysis

In [ ]:
# Standard imports
import sys
sys.path.insert(0, '../src')

import pandas as pd
import numpy as np
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from datetime import date, timedelta

# Volsurf imports
from volsurf.database.connection import get_connection
from volsurf.database.schema import init_schema
from volsurf.analytics import SurfaceMetrics, TermStructureAnalyzer, power_law

# Initialize
init_schema()
conn = get_connection()

SYMBOL = 'SPY'

## 1. Load Fitted Surface Data

In [ ]:
# Load all fitted surfaces
surfaces_df = conn.execute("""
    SELECT 
        quote_date,
        expiration_date,
        tte_years,
        svi_a, svi_b, svi_rho, svi_m, svi_sigma,
        atm_vol, skew_25delta,
        rmse, mae, max_error, num_points,
        passes_no_arbitrage
    FROM fitted_surfaces
    WHERE symbol = ?
    ORDER BY quote_date, expiration_date
""", [SYMBOL]).fetchdf()

print(f"Total fitted surfaces: {len(surfaces_df):,}")
print(f"Date range: {surfaces_df['quote_date'].min()} to {surfaces_df['quote_date'].max()}")
print(f"Trading days: {surfaces_df['quote_date'].nunique()}")

In [ ]:
# Add derived columns
surfaces_df['dte'] = (surfaces_df['tte_years'] * 365).round(0).astype(int)

# Summary statistics
print("\nSVI Parameter Statistics:")
param_cols = ['svi_a', 'svi_b', 'svi_rho', 'svi_m', 'svi_sigma']
surfaces_df[param_cols].describe().round(6)

## 2. Fit Quality Analysis

In [ ]:
# RMSE distribution
fig = make_subplots(rows=1, cols=2, subplot_titles=['RMSE Distribution', 'RMSE by DTE'])

fig.add_trace(
    go.Histogram(x=surfaces_df['rmse'] * 100, nbinsx=50, name='RMSE'),
    row=1, col=1
)

fig.add_trace(
    go.Scatter(
        x=surfaces_df['dte'], y=surfaces_df['rmse'] * 100,
        mode='markers', marker=dict(size=4, opacity=0.5), name='RMSE'
    ),
    row=1, col=2
)

fig.update_xaxes(title_text='RMSE (%)', row=1, col=1)
fig.update_xaxes(title_text='Days to Expiry', row=1, col=2)
fig.update_yaxes(title_text='RMSE (%)', row=1, col=2)

fig.update_layout(height=400, showlegend=False, title='Fit Quality Analysis')
fig.show()

print(f"\nFit Quality Summary:")
print(f"  Median RMSE: {surfaces_df['rmse'].median():.4%}")
print(f"  Mean RMSE: {surfaces_df['rmse'].mean():.4%}")
print(f"  95th percentile: {surfaces_df['rmse'].quantile(0.95):.4%}")
print(f"  % RMSE < 0.5%: {(surfaces_df['rmse'] < 0.005).mean():.1%}")

In [ ]:
# Fit quality over time
daily_rmse = surfaces_df.groupby('quote_date').agg({
    'rmse': ['mean', 'max'],
    'num_points': 'sum'
}).reset_index()
daily_rmse.columns = ['date', 'mean_rmse', 'max_rmse', 'total_points']

fig = go.Figure()

fig.add_trace(go.Scatter(
    x=daily_rmse['date'], y=daily_rmse['mean_rmse'] * 100,
    mode='lines', name='Mean RMSE'
))

fig.add_trace(go.Scatter(
    x=daily_rmse['date'], y=daily_rmse['max_rmse'] * 100,
    mode='lines', name='Max RMSE', line=dict(dash='dash')
))

fig.add_hline(y=0.5, line_dash='dot', line_color='red', annotation_text='0.5% target')

fig.update_layout(
    title='Fit Quality Over Time',
    xaxis_title='Date',
    yaxis_title='RMSE (%)',
    hovermode='x unified'
)

fig.show()

## 3. SVI Parameter Evolution

In [ ]:
# Select a tenor bucket (e.g., 25-35 DTE for ~30-day)
tenor_df = surfaces_df[(surfaces_df['dte'] >= 25) & (surfaces_df['dte'] <= 35)].copy()
print(f"Surfaces in 25-35 DTE bucket: {len(tenor_df)}")

In [ ]:
# Parameter evolution for fixed tenor
if len(tenor_df) > 0:
    daily_params = tenor_df.groupby('quote_date')[param_cols].mean().reset_index()
    
    fig = make_subplots(rows=2, cols=3, subplot_titles=['a (level)', 'b (wings)', 'rho (skew)', 
                                                        'm (shift)', 'sigma (smoothness)', 'ATM Vol'])
    
    fig.add_trace(go.Scatter(x=daily_params['quote_date'], y=daily_params['svi_a'], name='a'),
                  row=1, col=1)
    fig.add_trace(go.Scatter(x=daily_params['quote_date'], y=daily_params['svi_b'], name='b'),
                  row=1, col=2)
    fig.add_trace(go.Scatter(x=daily_params['quote_date'], y=daily_params['svi_rho'], name='rho'),
                  row=1, col=3)
    fig.add_trace(go.Scatter(x=daily_params['quote_date'], y=daily_params['svi_m'], name='m'),
                  row=2, col=1)
    fig.add_trace(go.Scatter(x=daily_params['quote_date'], y=daily_params['svi_sigma'], name='sigma'),
                  row=2, col=2)
    
    # ATM vol
    daily_atm = tenor_df.groupby('quote_date')['atm_vol'].mean().reset_index()
    fig.add_trace(go.Scatter(x=daily_atm['quote_date'], y=daily_atm['atm_vol']*100, name='ATM Vol'),
                  row=2, col=3)
    
    fig.update_layout(height=600, showlegend=False, title='SVI Parameter Evolution (~30 DTE)')
    fig.show()

In [ ]:
# Skew (rho) vs ATM level
if len(tenor_df) > 0:
    fig = px.scatter(
        tenor_df,
        x='atm_vol',
        y='svi_rho',
        color='quote_date',
        title='Skew vs ATM Vol (~30 DTE)',
        labels={'atm_vol': 'ATM Vol', 'svi_rho': 'Rho (Skew)', 'quote_date': 'Date'}
    )
    fig.update_xaxes(tickformat='.1%')
    fig.show()
    
    # Correlation
    corr = tenor_df[['atm_vol', 'svi_rho']].corr().iloc[0, 1]
    print(f"Correlation between ATM vol and skew: {corr:.3f}")

## 4. Term Structure Analysis

In [ ]:
# Term structure on latest date
latest_date = surfaces_df['quote_date'].max()
latest_surfaces = surfaces_df[surfaces_df['quote_date'] == latest_date].copy()

print(f"Term structure on {latest_date}:")
print(f"  Expirations: {len(latest_surfaces)}")

In [ ]:
# Plot ATM term structure with power law fit
analyzer = TermStructureAnalyzer()
result = analyzer.analyze_date(SYMBOL, latest_date)

fig = go.Figure()

# Actual ATM vols
fig.add_trace(go.Scatter(
    x=latest_surfaces['dte'], y=latest_surfaces['atm_vol'] * 100,
    mode='markers+lines', name='ATM Vol',
    marker=dict(size=10)
))

# Power law fit
if result and result.atm_fit:
    tte_smooth = np.linspace(latest_surfaces['tte_years'].min(), 
                              latest_surfaces['tte_years'].max(), 100)
    vol_fit = power_law(tte_smooth, result.atm_fit.a, result.atm_fit.b)
    
    fig.add_trace(go.Scatter(
        x=tte_smooth * 365, y=vol_fit * 100,
        mode='lines', name=f'Power Law (a={result.atm_fit.a:.4f}, b={result.atm_fit.b:.4f})',
        line=dict(dash='dash', color='red')
    ))

fig.update_layout(
    title=f'{SYMBOL} ATM Vol Term Structure ({latest_date})',
    xaxis_title='Days to Expiry',
    yaxis_title='ATM Vol (%)',
    hovermode='x unified'
)

fig.show()

In [ ]:
# Skew term structure
fig = go.Figure()

fig.add_trace(go.Scatter(
    x=latest_surfaces['dte'], y=latest_surfaces['svi_rho'],
    mode='markers+lines', name='Rho (Skew)',
    marker=dict(size=10)
))

fig.add_hline(y=0, line_dash='dash', line_color='gray')

fig.update_layout(
    title=f'{SYMBOL} Skew Term Structure ({latest_date})',
    xaxis_title='Days to Expiry',
    yaxis_title='Rho (Skew)',
    hovermode='x unified'
)

fig.show()

## 5. Parameter Stability Analysis

In [ ]:
# Calculate day-over-day parameter changes for fixed tenor
if len(tenor_df) > 1:
    daily_params = tenor_df.groupby('quote_date')[param_cols + ['atm_vol']].mean().reset_index()
    daily_params = daily_params.sort_values('quote_date')
    
    # Calculate changes
    for col in param_cols + ['atm_vol']:
        daily_params[f'd_{col}'] = daily_params[col].diff()
    
    print("Day-over-Day Parameter Changes (30 DTE bucket):")
    change_cols = [f'd_{col}' for col in param_cols + ['atm_vol']]
    print(daily_params[change_cols].describe().round(6))

In [ ]:
# Parameter stability - coefficient of variation
if len(tenor_df) > 0:
    stability = {}
    for col in param_cols:
        mean = tenor_df[col].mean()
        std = tenor_df[col].std()
        cv = std / abs(mean) if mean != 0 else np.nan
        stability[col] = {'mean': mean, 'std': std, 'CV': cv}
    
    stability_df = pd.DataFrame(stability).T
    stability_df = stability_df.round(4)
    print("\nParameter Stability (Coefficient of Variation):")
    print(stability_df)
    print("\nLower CV = more stable parameter")

## 6. Arbitrage Checks

In [ ]:
# Arbitrage violation summary
if 'passes_no_arbitrage' in surfaces_df.columns:
    arb_summary = surfaces_df.groupby('quote_date').agg({
        'passes_no_arbitrage': ['sum', 'count']
    }).reset_index()
    arb_summary.columns = ['date', 'arb_free', 'total']
    arb_summary['pct_arb_free'] = arb_summary['arb_free'] / arb_summary['total']
    
    print(f"Arbitrage-Free Surfaces:")
    print(f"  Overall: {surfaces_df['passes_no_arbitrage'].sum()}/{len(surfaces_df)} ({surfaces_df['passes_no_arbitrage'].mean():.1%})")
    
    fig = px.line(
        arb_summary,
        x='date', y='pct_arb_free',
        title='Percentage of Arbitrage-Free Fits Over Time',
        labels={'pct_arb_free': '% Arb-Free', 'date': 'Date'}
    )
    fig.update_yaxes(tickformat='.0%')
    fig.show()
else:
    print("Arbitrage check data not available")

## Summary

Key findings from model analysis:

1. **Fit Quality**: [Add observations]
2. **Parameter Stability**: [Add observations]
3. **Term Structure**: [Add observations]
4. **Arbitrage**: [Add observations]